In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
import xml.etree.ElementTree as ET

#For model
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#For shwing images and graphs
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [1]:
classes = ('cloth_mask', 'mask_worn_incorrectly', 'n95_mask', 'no_mask', 'surgical_mask')

In [ ]:
imagePath = 'Classes/'
transform = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()])#Just resizing the image to 128*128 size

data = datasets.ImageFolder(root=imagePath, transform=transform)

loader = DataLoader(dataset=data,batch_size=32)

In [ ]:
#Image size normalization

def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        #print(len(data))
        #print(data)
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [ ]:
mean, std = get_mean_and_std(loader)
print('Before normalization:')
print('Mean: '+  str(mean))
print('Standard Dev: '+  str(std))

In [ ]:
transform = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor(),
                                transforms.Normalize(mean=mean,std=std)])
data = datasets.ImageFolder(root=imagePath, transform=transform)


loader = DataLoader(data,batch_size=32)
new_mean, new_std = get_mean_and_std(loader)
print('After normalization:')
print('Mean: '+  str(new_mean))
print('Standard Dev: '+  str(new_std))

In [ ]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size], generator=torch.Generator().manual_seed(42))
#print('Train size: ' + str(len(train_dataset)))
#print('Test size: ' + str(len(test_dataset)))
train_loader = DataLoader(train_dataset,batch_size=32)
validation_loader = DataLoader(data,batch_size=32)
test_loader = DataLoader(test_dataset,batch_size=32)

# #Delete this later
# batch =next(iter(validation_loader))
# images,labels=batch
# grid=torchvision.utils.make_grid(images,nrow=3)
# plt.figure(figsize=(11,11))
# plt.imshow(np.transpose(grid,(1,2,0)))
# print('labels',labels)